# Speeding up Python Code

In this notebook we will explore a few examples comparing different approaches for speeding up python code. We consider a function that calculates the pairwise distance between M points in a N dimensional vector space. This is a nice test function since computing distances is amongst the most common array-based operations in many fields.

## Python

In [ ]:
import numpy

X = numpy.random.random((1000, 3))

def pairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = numpy.empty((M, M))
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = numpy.sqrt(d)
    return D

%timeit pairwise_python(X)

## Numpy

In [ ]:
def pairwise_numpy(X):
    return numpy.sqrt(((X[:, None, :] - X) ** 2).sum(-1))

%timeit pairwise_numpy(X)

## Numba

[Numba](http://numba.pydata.org) is a library that uses just-in-time compiling to speed up computationally intensive python code. Most functionality is to just add decorators to standard python functions

In [ ]:
import numba

pairwise_numba = numba.jit(pairwise_python)

%timeit pairwise_numba(X)

## Cython

Cython is a programming language that is a superset of python. It allows calling of C functions and declaring C types. Cython code is then compiled to very efficient C code which executes faster than standard python

In [ ]:
%load_ext Cython

In [ ]:
%%cython
import numpy
cimport cython
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def pairwise_cython(double[:, ::1] X):
    cdef int M = X.shape[0]
    cdef int N = X.shape[1]
    cdef double tmp, d
    cdef double[:, ::1] D = numpy.empty((M, M), dtype=numpy.float64)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = sqrt(d)
    return numpy.asarray(D)

In [ ]:
%timeit pairwise_numba(X)